In [1]:
import gym
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
import os
import numpy as np
import math
import random
import matplotlib as plt
import random


env = gym.make("CartPole-v1")
env.reset()


2022-03-18 11:21:47.639596: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-18 11:21:47.639616: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


array([-0.00627525, -0.03542977,  0.04383823,  0.02772941], dtype=float32)

In [2]:
in_shape = (env.observation_space.shape[0] + 1)
print(in_shape)

5


In [3]:
class Agent: 
    def __init__(self, state_size, action_size):
        self.weight_backup      = "cartpole_weight.h5"
        self.state_size         = state_size
        self.action_size        = action_size

        self.store              = []
        self.memory             = []
        self.learning_rate      = 0.001
        self.gamma              = 0.95
        self.exploration_rate   = 1.0
        self.exploration_min    = 0.01
        self.exploration_decay  = 0.999
        self.model              = self._build_model()
        
    def _build_model(self):
        #input: state
        inputs = keras.Input(shape=self.state_size)
        dense = layers.Dense(16)
        x = dense(inputs)
        x = layers.Dense(64, activation="relu")(x)
        x = layers.Dense(64, activation="relu")(x)
        x = layers.Dense(32, activation="relu")(x)
        #output excpected reward
        outputs = layers.Dense(self.action_size, activation = "linear")(x)
        model = keras.Model(inputs=inputs, outputs=outputs, name="cartpole-model")  
        model.compile(optimizer="Adam", loss="mse")
        model.summary()
        return model
    
    def update_memory(self, step):
        #Format of step: [state(t), reward(t+1), action(t), state(t+1), done?]
        self.memory.append(step)
        
    def take_action(self, state):
        act_rew = {}
        
        
        """for i in range(self.action_size):
            act_rew[i] = self.predict(state, i)
        """    
        #action with best score
        scores = self.predict(state)[0]
        #print(scores)
        action = np.argmax(scores)#max(act_rew, key=act_rew.get)
        
        if (random.random() <= self.exploration_rate):
            
            action = random.randint(0, self.action_size - 1)
            #choose random exploration action
            #action = random.choice(list(act_rew))
            
        
        return action
    
    def predict(self, state):
        return self.model.predict(state.reshape(1, self.state_size))
    
    
    def replay(self, epochs = 1, sample_size = 100):
        global debug_vars
        
        debug_vars = []
        #replay with memory
        #calculate rewards
        
        #Bellman equation for infinite MDP's
        #Q(s, a) = Q(s, a) + alpha(R(s, a) + y*(maxQ(s', a) - Q(s, a))
        if len(self.memory) < 20:
            return
        if len(self.memory) < sample_size:
            sample_size = len(self.memory)
        sample_batch = random.sample(self.memory, sample_size)
        
      
        for state, action, reward, next_state, done in sample_batch:
            
            state = np.reshape(state, (1,4))
            next_state = np.reshape(next_state, (1,4))
            target = reward
            if not done:
              target = reward + self.gamma * np.amax(self.model.predict(next_state)[0])      
            target_prediction = self.model.predict(state)
            target_prediction[0][action] = target

            #Training
            #print(state)
            self.model.fit(state, target_prediction, epochs=1, verbose=0)
            
                    #exploration (declinining with t)

        if self.exploration_rate >= self.exploration_min:
            self.exploration_rate *= self.exploration_decay
            
        
 
    def save_model(self):
        self.model.save("proto.h5")
        
    def load_model(self, name = "proto.h5"):
        self.model = keras.models.load_model(name)

In [4]:
agent = Agent(env.observation_space.shape[0], env.action_space.n)

Model: "cartpole-model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4)]               0         
                                                                 
 dense (Dense)               (None, 16)                80        
                                                                 
 dense_1 (Dense)             (None, 64)                1088      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 2)                 66        
                                                                 
Total params: 7,474
Trainable params: 7,474
Non-trai

2022-03-18 11:21:51.277536: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-18 11:21:51.277557: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-18 11:21:51.277571: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (topas): /proc/driver/nvidia/version does not exist
2022-03-18 11:21:51.277724: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
agent.load_model("proto.h5")

In [ ]:
while True:
    
    observation = env.reset()
    #env.render()
    index = 0
    
    while True:
        
        action = agent.take_action(observation)
        #print(next_action)
        #print(next_action)
        next_observation, reward, done, _ = env.step(action)
        
        step_rem = [observation, action, reward, next_observation, done]
    
        agent.update_memory(step_rem)
        
        
        #switch to next state
        observation = next_observation
        
        if done:
            agent.replay()
            print(f"e: {agent.exploration_rate}")
            print(f"Done after {index} steps")
            print("------------------------")
            break
        else:    
            index += 1 
    if index > 2000:
        print("DONE")
        break
            
env.close()

e: 1.0
Done after 14 steps
------------------------
e: 0.999
Done after 18 steps
------------------------
e: 0.998001
Done after 11 steps
------------------------
e: 0.997002999
Done after 18 steps
------------------------
e: 0.996005996001
Done after 19 steps
------------------------
e: 0.995009990004999
Done after 11 steps
------------------------
e: 0.994014980014994
Done after 14 steps
------------------------
e: 0.993020965034979
Done after 11 steps
------------------------
e: 0.9920279440699441
Done after 8 steps
------------------------
e: 0.9910359161258742
Done after 28 steps
------------------------
e: 0.9900448802097482
Done after 18 steps
------------------------
e: 0.9890548353295385
Done after 27 steps
------------------------
e: 0.988065780494209
Done after 21 steps
------------------------
e: 0.9870777147137147
Done after 20 steps
------------------------
e: 0.986090636999001
Done after 9 steps
------------------------
e: 0.9851045463620021
Done after 66 steps
---------

e: 0.8833250364922639
Done after 12 steps
------------------------
e: 0.8824417114557717
Done after 15 steps
------------------------
e: 0.8815592697443159
Done after 16 steps
------------------------
e: 0.8806777104745716
Done after 9 steps
------------------------
e: 0.8797970327640969
Done after 10 steps
------------------------
e: 0.8789172357313328
Done after 12 steps
------------------------
e: 0.8780383184956015
Done after 18 steps
------------------------
e: 0.8771602801771059
Done after 9 steps
------------------------
e: 0.8762831198969288
Done after 9 steps
------------------------
e: 0.8754068367770318
Done after 35 steps
------------------------
e: 0.8745314299402548
Done after 16 steps
------------------------
e: 0.8736568985103146
Done after 21 steps
------------------------
e: 0.8727832416118043
Done after 37 steps
------------------------
e: 0.8719104583701925
Done after 40 steps
------------------------
e: 0.8710385479118223
Done after 22 steps
-----------------------

In [ ]:
agent.model.save("proto.h5")

In [ ]:
env.action_space